In [6]:
import sys
import argparse
import warnings
from pathlib import Path
from scripts.processing import Processor
from scripts.folder_structure import FolderStructure
from scripts.analyze import load_pyseer_hits
from scripts.analyze import AnalyzeResults
from scripts.figures import get_k_loci, FIGURE4_PANELA, FIGURE4_PANELB, FIGURE4_PANELC

%load_ext autoreload
%autoreload 2

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 100)

# workflow IO
input_dir = "/Users/januszkoszucki/MGG Dropbox/Projects/kleb-prophage-div/2025-02-12_KLEBDATA_LIGHT"
output_dir = "/Users/januszkoszucki/MGG Dropbox/Projects/kleb-prophage-div/2025-02-12_KLEBDATA_LIGHT/3_GWAS/2025-05-13_KLEBDATA_LIGHT_GWAS"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# folder structure
folder_manager = FolderStructure(input_dir, output_dir)
structure = folder_manager.get_paths_dict()
params = folder_manager.get_params_dict()

# aggregate results
processor = Processor(structure, params)
processor.aggregate_pyseer_hits()
processor.aggregate_function_prediction()

# load
pyseer_hits = load_pyseer_hits(structure, params)

Created directory: /Users/januszkoszucki/tmp_blast_dir


In [9]:
analyze = AnalyzeResults(structure, params)

analyze.per_locus_analysis(run=False)
analyze.aggregate_data(run=True)
analyze.combine_predictions_and_enzymes(run=True)

Error creating symbolic link: [Errno 17] File exists: '../../1_INTERMEDIATE/1_PROCESSED_INPUT/PROTMINLEN300_SPECIES-KPN-KVV-KQQ-KQS_MINNCONTIGS500/2_FUNCTIONS' -> '/Users/januszkoszucki/MGG Dropbox/Projects/kleb-prophage-div/2025-02-12_KLEBDATA_LIGHT/3_GWAS/2025-05-13_KLEBDATA_LIGHT_GWAS/4_ANALYZE/2_AGGREGATED_DATA/2_FUNCTIONS'
Error creating symbolic link: [Errno 17] File exists: '../../1_INTERMEDIATE/1_PROCESSED_INPUT/PROTMINLEN300_SPECIES-KPN-KVV-KQQ-KQS_MINNCONTIGS500/1_PROTEINS' -> '/Users/januszkoszucki/MGG Dropbox/Projects/kleb-prophage-div/2025-02-12_KLEBDATA_LIGHT/3_GWAS/2025-05-13_KLEBDATA_LIGHT_GWAS/4_ANALYZE/2_AGGREGATED_DATA/1_PROTEINS'


In [35]:
FIGURE4_PANELA(structure, params)
FIGURE4_PANELB(structure, params)
FIGURE4_PANELC(structure, params)



Building a new DB, current time: 05/15/2025 11:03:55
New DB name:   /Users/januszkoszucki/tmp_blast_dir/sequences.fasta
New DB title:  /Users/januszkoszucki/tmp_blast_dir/sequences.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 144 sequences in 0.00282192 seconds.




In [15]:
structure.keys()


dict_keys(['input_processed', 'mmseqs', 'functions', 'gwas', 'processed', 'analyze', 'figures', 'input'])

In [ ]:
pd.read_csv()